# M11.1 BRIDGE: Tenant Isolation → Tenant Customization

**Track:** CCC Level 3  
**Module:** 11 (Multi-Tenant SaaS Architecture)  
**Type:** Within-Module Bridge  
**Previous:** M11.1 Augmented (Tenant Isolation Implementation)  
**Next:** M11.2 Concept (Tenant-Specific Customization)

---

## Purpose

This notebook validates readiness to move from M11.1 (Tenant Isolation) to M11.2 (Tenant Customization).  
It checks the four critical prerequisites defined in the bridge script.

---

## Section 1: RECAP — What M11.1 Shipped

### Accomplishments in M11.1 Tenant Isolation:

✓ **Multi-tenant isolation system**  
   PostgreSQL Row-Level Security (RLS) preventing cross-tenant data leakage even with application bugs

✓ **Namespace-based data separation**  
   Each tenant's vectors isolated in dedicated Pinecone namespaces, supporting 100 tenants per index

✓ **Cost allocation engine**  
   Per-tenant cost tracking with monthly reconciliation (allocated costs within 5% of actual bills)

✓ **Auto-scaling logic**  
   Automatic provisioning of new Pinecone indexes when namespace capacity reaches 80% (90/100 namespaces)

### What This Enables:

Production-grade isolation for 100 customers with confidence that:
- Their data stays separate
- You can track costs per tenant for accurate pricing

In [ ]:
# Quick verification: Import libraries for readiness checks
import os
import json
from datetime import datetime

print("✓ Bridge readiness notebook initialized")
print(f"Timestamp: {datetime.now().isoformat()}")

# Expected:
# ✓ Bridge readiness notebook initialized
# Timestamp: 2025-11-09T...

---

## Section 2: Readiness Check #1 — Row-Level Security Policies

### Verification Requirement:

☐ **Row-Level Security policies verified**  
   → Check: Query from Tenant A with wrong tenant_id returns zero results (not error, but empty set)  
   → Impact: Saves 8 hours debugging data leakage in production

### Test Criteria:

1. Attempt to query Tenant A's data using Tenant B's credentials
2. Expected result: Empty result set (0 rows), not an error
3. Pass: No cross-tenant data leakage

In [ ]:
# RLS Verification Check (stub - requires PostgreSQL connection)

# Check if database credentials exist
DB_CONNECTION_STRING = os.getenv("DATABASE_URL") or os.getenv("POSTGRES_URL")

if not DB_CONNECTION_STRING:
    print("⚠️ Skipping RLS check (no DATABASE_URL configured)")
    print("To run: Set DATABASE_URL and install psycopg2")
else:
    # Stub for actual RLS verification
    # Would connect to DB and test cross-tenant query
    print("✓ Database credentials found")
    print("TODO: Implement RLS verification:")
    print("  1. Connect as Tenant A")
    print("  2. Query Tenant B's data")
    print("  3. Assert: result.rowcount == 0")

# Expected:
# ⚠️ Skipping RLS check (no DATABASE_URL configured)
# OR
# ✓ Database credentials found
# TODO: Implement RLS verification...

---

## Section 3: Readiness Check #2 — Cost Allocation Reconciliation

### Verification Requirement:

☐ **Cost allocation reconciliation passing**  
   → Check: Run monthly report—allocated costs match actual bills within ±5%  
   → Impact: Prevents $2,000-5,000/month pricing errors from untracked costs

### Test Criteria:

1. Compare allocated costs (sum of all tenant costs) to actual API bills
2. Acceptable variance: ±5%
3. **Warning:** If variance > 10%, STOP and fix cost tracking before M11.2

### Why This Matters:

Without accurate cost allocation, you can't:
- Price tenant tiers correctly
- Identify which tenants are profitable
- Optimize per-tenant configurations in M11.2

In [ ]:
# Cost Reconciliation Check (stub - requires cost tracking data)

# Placeholder: Sample cost reconciliation
# In production, this would query your cost tracking database

sample_costs = {
    "allocated_total": 4750.00,  # Sum of all tenant costs
    "actual_openai_bill": 4800.00,  # From OpenAI dashboard
    "actual_pinecone_bill": 150.00,  # From Pinecone dashboard
}

if not os.path.exists("cost_reports"):
    print("⚠️ Skipping cost check (no cost_reports/ directory)")
    print("To run: Create cost_reports/ with monthly CSV files")
else:
    allocated = sample_costs["allocated_total"]
    actual = sample_costs["actual_openai_bill"] + sample_costs["actual_pinecone_bill"]
    variance_pct = abs((allocated - actual) / actual) * 100
    
    print(f"Allocated: ${allocated:.2f}")
    print(f"Actual: ${actual:.2f}")
    print(f"Variance: {variance_pct:.1f}%")
    
    if variance_pct <= 5:
        print("✓ PASS: Within ±5% threshold")
    elif variance_pct <= 10:
        print("⚠️ WARNING: 5-10% variance - review tracking")
    else:
        print("❌ FAIL: >10% variance - fix before M11.2")

# Expected:
# ⚠️ Skipping cost check (no cost_reports/ directory)

---

## Section 4: Readiness Check #3 — Namespace Capacity Monitoring

### Verification Requirement:

☐ **Namespace capacity monitoring configured**  
   → Check: Alert fires at 72/90 namespaces used (80% threshold)  
   → Impact: Prevents hitting namespace limit and blocking new tenant signups

### Test Criteria:

1. Verify monitoring system tracks namespace usage per Pinecone index
2. Alert threshold set at 80% (72 out of 90 namespaces)
3. Auto-scaling triggered before hitting 90/90 limit

### Why This Matters:

Pinecone indexes support 100 namespaces max. At 90/100:
- New tenant signups will fail
- No time to provision new index (can take 10-15 minutes)
- Revenue loss during downtime

In [ ]:
# Namespace Capacity Check (stub - requires Pinecone API key)

PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")

if not PINECONE_API_KEY:
    print("⚠️ Skipping namespace check (no PINECONE_API_KEY)")
    print("To run: Set PINECONE_API_KEY and install pinecone-client")
else:
    # Stub for actual namespace monitoring
    # Would query Pinecone API for namespace counts
    print("✓ Pinecone API key found")
    print("TODO: Implement namespace monitoring:")
    print("  1. List all indexes")
    print("  2. For each index, count namespaces")
    print("  3. Alert if any index >= 72 namespaces")
    
    # Sample monitoring config (JSON format)
    monitoring_config = {
        "namespace_threshold": 72,
        "max_namespaces": 90,
        "alert_channel": "slack_webhook_url",
        "autoscale_enabled": True
    }
    print(f"\nMonitoring config: {json.dumps(monitoring_config, indent=2)}")

# Expected:
# ⚠️ Skipping namespace check (no PINECONE_API_KEY)

---

## Section 5: Readiness Check #4 — Test Tenant Provisioning

### Verification Requirement:

☐ **At least 3 test tenants provisioned**  
   → Check: Can create tenant, upload documents, query successfully for 3 separate tenant IDs  
   → Impact: Provides baseline for testing per-tenant configurations in M11.2

### Test Criteria:

1. Verify 3+ tenants exist in the system
2. Each tenant should have:
   - Unique tenant_id
   - At least 1 document uploaded
   - Successful query/retrieval capability
3. Tenants should have different data characteristics for M11.2 testing

### PractaThon Data (from bridge script):

For M11.2 readiness, you should have:

| Tenant ID | Doc Count | Avg Doc Size | Use Case |
|-----------|-----------|--------------|----------|
| tenant_d | 1,000 | 150 words | High-volume, small docs (customer support) |
| tenant_e | 50 | 5,200 words | Low-volume, large docs (legal/research) |
| tenant_f | 200 | 800 words | Mid-range (general business docs) |

This diversity enables testing different model/configuration strategies in M11.2.

In [ ]:
# Test Tenant Verification (stub - requires tenant database)

# Check if tenant registry exists
TENANT_REGISTRY = "tenants.json"  # Could be DB or config file

if not os.path.exists(TENANT_REGISTRY):
    print("⚠️ Skipping tenant check (no tenants.json found)")
    print("Creating sample tenant registry for reference...")
    
    # Sample tenant data structure
    sample_tenants = [
        {
            "tenant_id": "tenant_d",
            "name": "Customer Support Co",
            "doc_count": 1000,
            "avg_doc_size_words": 150,
            "use_case": "High-volume customer support"
        },
        {
            "tenant_id": "tenant_e", 
            "name": "Legal Research LLC",
            "doc_count": 50,
            "avg_doc_size_words": 5200,
            "use_case": "Legal documents and case law"
        },
        {
            "tenant_id": "tenant_f",
            "name": "Business Docs Inc",
            "doc_count": 200,
            "avg_doc_size_words": 800,
            "use_case": "General business documentation"
        }
    ]
    
    print(f"Sample tenant registry (3 tenants):")
    for t in sample_tenants:
        print(f"  - {t['tenant_id']}: {t['doc_count']} docs")
    
    print("\n✓ 3 diverse tenants configured for M11.2 testing")
else:
    # Would read actual tenant registry
    print(f"✓ Tenant registry found at {TENANT_REGISTRY}")
    print("TODO: Validate tenant provisioning:")
    print("  1. Load tenant list")
    print("  2. Verify >= 3 tenants exist")
    print("  3. Test query for each tenant")

# Expected:
# ⚠️ Skipping tenant check (no tenants.json found)
# Creating sample tenant registry for reference...
# Sample tenant registry (3 tenants):
#   - tenant_d: 1000 docs
#   - tenant_e: 50 docs
#   - tenant_f: 200 docs
# ✓ 3 diverse tenants configured for M11.2 testing

---

## Section 6: CALL-FORWARD — What M11.2 Will Introduce

### The Next Challenge:

**Current State (M11.1):**  
Every tenant runs identical configuration (same model, same parameters, same costs).

**The Problem:**  
- Enterprise customers need GPT-4 and high accuracy ($5,000/month budget)
- Small businesses need fast responses with GPT-3.5 ($500/month budget)
- Both currently get the same service → waste money OR lose customers

### M11.2 Tenant Customization Will Add:

**1. Per-Tenant Model Selection**
- Different tenants can use different LLMs (GPT-4, GPT-3.5, Claude)
- Based on their tier and needs
- Example: Law firm uses GPT-4, customer support uses GPT-3.5

**2. Database-Driven Configuration System**
- Add/modify tenant configs via database UPDATE
- Zero code deployments
- Changes take effect immediately

**3. Safe Prompt Template Injection**
- Let tenants customize prompts while preventing prompt injection attacks
- Validation and sandboxing for security
- Maintain system integrity

### The Key Question for M11.2:

**"How do you let each tenant customize their experience without turning your codebase into an unmaintainable mess?"**

### Cost Optimization Example:

| Tenant Type | Current Cost/Query | M11.2 Optimized | Monthly Savings |
|-------------|-------------------|-----------------|-----------------|
| Customer Support | $0.045 | $0.008 | $185/month |
| Legal Research | $0.045 | $0.045 | $0 (needs premium) |
| General Business | $0.045 | $0.020 | $125/month |

**Across 100 tenants:** Potential savings of $18,500/month in API costs.

---

## Ready for M11.2?

If all four readiness checks passed (or you understand what to fix), you're ready to proceed to **M11.2 Concept + Augmented**.

In [ ]:
# Bridge Validation Summary

print("=" * 60)
print("M11.1 → M11.2 BRIDGE VALIDATION COMPLETE")
print("=" * 60)
print("\nReadiness Checklist:")
print("☐ RLS policies verified")
print("☐ Cost reconciliation passing (±5%)")
print("☐ Namespace monitoring configured (80% threshold)")
print("☐ 3+ test tenants provisioned")
print("\nNext Step: Proceed to M11.2 Concept + Augmented")
print("Focus: Tenant-specific customization and configuration")
print("=" * 60)

# Expected:
# ============================================================
# M11.1 → M11.2 BRIDGE VALIDATION COMPLETE
# ============================================================
# Readiness Checklist:
# ☐ RLS policies verified
# ☐ Cost reconciliation passing (±5%)
# ☐ Namespace monitoring configured (80% threshold)
# ☐ 3+ test tenants provisioned
# Next Step: Proceed to M11.2 Concept + Augmented
# Focus: Tenant-specific customization and configuration
# ============================================================